In [0]:
dbutils.widgets.text('marketplace', '')
marketplace = dbutils.widgets.get('marketplace')


A_KEY = 'AKIAW.....GXH2GE'
A_SECRET = 'Kz54x......zb20vBx'
ARN = 'arn:aws:iam::466...117:role/e..ngapiRole'


C_KEY = 'amzn1.application-oa2-client.c895cc.......31cee69f'
C_SECRET_KEY = '712278808........8ed55a46827d1d09c76'
C_REFRESH_TOKEN = 'Atzr|IwEBIDL............09E0nLwVypkXhs'


if marketplace == 'DE':
    MarketplaceID = 'A1PA6795UKMFR9'
    
elif  marketplace == 'IT':
    MarketplaceID = 'APJ6JRA9NG5V4'
    
elif marketplace == 'FR':
    MarketplaceID = 'A13V1IB3VIYZZH'
    
elif marketplace == 'NL':
     MarketplaceID = 'A1805IZSGTT6HS'
        
elif marketplace == 'ES':
     MarketplaceID = 'A1RKKUPIHCS9HS'
        
elif marketplace == 'UK':
     MarketplaceID = 'A1F83G8C2ARO7P'   
        
elif marketplace == 'SE':
     MarketplaceID = 'A2NODRKZP88ZB9'   

elif marketplace == 'PL':
     MarketplaceID = 'A1C3SOZRARQ6R3'   

In [0]:
from time import sleep
import requests
import re
import json
import datetime
import gzip
from requests_aws4auth import AWS4Auth



def get_date():
    return datetime.datetime.utcnow().strftime("%Y%m%d")



def get_access_token():
    global C_KEY, C_SECRET_KEY, C_REFRESH_TOKEN
    r = requests.post("https://api.amazon.com/auth/o2/token",
                      data={
                            "grant_type":"refresh_token",
                            "refresh_token":C_REFRESH_TOKEN,
                            "client_id": C_KEY,
                            "client_secret": C_SECRET_KEY
                            })

    return r.json()['access_token']




def assume_role():
    r = requests.get( f"https://sts.amazonaws.com?Version=2011-06-15&Action=AssumeRole&RoleSessionName=Test&RoleArn={ARN}&DurationSeconds=3600", 
                      auth = AWS4Auth(A_KEY, A_SECRET, 'us-east-1', 'sts') )
    
    return (re.findall('<AccessKeyId>(.+)</AccessKeyId>', r.text)[0], re.findall('<SecretAccessKey>(.+)</SecretAccessKey>', r.text)[0], re.findall('<SessionToken>(.+)</SessionToken>', r.text)[0])



# Get the Access Keys
access_token = get_access_token()
access_key, secret_key, session_token = assume_role()


# Europe endpoint : https://sellingpartnerapi-eu.amazon.com
# Europe AWS region : eu-west-1

# This request needs parameters. Use the 'params' in the request.
# For requests that needs body information, use 'data' in the request.
 


def get_data(related_url, parameters):
    global access_token, access_key, secret_key, session_token

    req = requests.get(
                       f"https://sellingpartnerapi-eu.amazon.com{related_url}",
                       params = parameters,
                       headers = {'x-amz-access-token': access_token, 'Content-Type': 'application/x-www-form-urlencoded'},
                       auth = AWS4Auth(access_key,
                                       secret_key,
                                       'eu-west-1',
                                       'execute-api',
                                       session_token = session_token)  )
    return(req.json())




# Define related url and parameters, put the each responses to mounted container, get nextToken from previous result
# When a response exceeds the pageSize, pagination occurs. This means the response is divided into individual pages.
# To retrieve the next page or the previous page, you must pass the nextToken value or the previousToken value as the "nextToken" parameter in the next request.
def get_data_all(nextToken, id,  fileName):
    global MarketplaceID

    if nextToken is None:
        req = get_data('/fba/inventory/v1/summaries', { 'details'          : 'true',
                                                        'granularityType'  : 'Marketplace',
                                                        'granularityId'    :  MarketplaceID,
                                                        'marketplaceIds'   :  MarketplaceID } )
    else:
        req = get_data('/fba/inventory/v1/summaries', { 'nextToken'        :  nextToken,
                                                        'details'          : 'true',
                                                        'granularityType'  : 'Marketplace',
                                                        'granularityId'    :  MarketplaceID,
                                                        'marketplaceIds'   :  MarketplaceID } )
        
    print('dbfs:/mnt/' + containerName + '/' + f'{fileName}_{id}.json')
    dbutils.fs.put('dbfs:/mnt/' + containerName + '/' + f'{fileName}_{id}.json', json.dumps(req, indent=4), overwrite = True)
    print('Done: ', id)
    try:
        return (req["pagination"]["nextToken"])
    except:
        return None
    
    
    
    
# Retrieve all data recursively until there is no pagination nextToken. 
def download_data_all(fileName):
    global access_token, access_key, secret_key, session_token
    i = 1
    nexttoken = get_data_all(None, 1, fileName)
    
    while nexttoken is not None:
        i+=1
        nexttoken = get_data_all(nexttoken, i, fileName=fileName)
        if i % 30 == 0:
            print('sleeping')
            sleep(20)
        if i % 500 == 0:
            print('Refreshing Keys...')
            access_token = get_access_token()
            access_key, secret_key, session_token = assume_role()
            print('Keys Refreshed!')


In [ ]:
# Mounting to the blob container
containerName = "amazon-inventory"
storageAccountName = "blobinventory"
sas = "sp=racwdli&st=2022-08-1Hx9.............R5JY%3D"
config = "fs.azure.sas." + containerName+ "." + storageAccountName + ".blob.core.windows.net"

try:
    dbutils.fs.unmount("/mnt/amazon-inventory")
except:
    pass
    
try:
    dbutils.fs.mount(
      source = f"wasbs://{containerName}@{storageAccountName}.blob.core.windows.net",
      mount_point = "/mnt/amazon-inventory",
      extra_configs = {config : sas})
except:
    pass

In [ ]:
# Download Inventory data into blob
current_day = get_date()
download_data_all(f"{current_day}_DE")